In [ ]:
import os
import json
import pandas as pd
from ultralytics import YOLO


In [16]:
# --- CONFIGURATION ---
dataset_dir = "Data" 

# Chargement du modèle
# On utilise le modèle nano (léger) standard. 
# Il est entraîné sur COCO, donc on détectera la classe 0 ("person") qui correspond à la main/bras.
model = YOLO('yolov8n.pt') 

# Structures de données pour l'export 
ei_labels = {
    "version": 1,
    "type": "bounding-box",
    "boundingBoxes": {}
}

# Format CSV (pour vérification manuelle)
csv_data = []

print(f"--- Analyse du dossier '{dataset_dir}' ---")

if not os.path.exists(dataset_dir):
    print(f"ERREUR: Le dossier '{dataset_dir}' est introuvable.")
    exit()

# Parcours des dossiers de classes (A.class, B.class, etc.)
for class_folder in sorted(os.listdir(dataset_dir)):
    folder_path = os.path.join(dataset_dir, class_folder)
    
    if not os.path.isdir(folder_path):
        continue
        
    # --- 1. NETTOYAGE DU LABEL ---
    # Transforme "A.class" en "A"
    if class_folder.endswith(".class"):
        label = class_folder.replace(".class", "")
    else:
        label = class_folder # Cas où le dossier serait déjà nommé "A"
    
    print(f"Traitement du dossier : {class_folder} -> Label : {label}")
    
    # Parcours des images (00000.jpg, 00001.jpg...)
    for image_file in os.listdir(folder_path):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
            
        full_path = os.path.join(folder_path, image_file)
        
        # --- IMPORTANT : GESTION DES DOUBLONS ---
        # Comme vos fichiers s'appellent tous 00000.jpg dans chaque dossier,
        # nous utilisons le chemin relatif (ex: "A.class/00000.jpg") comme clé unique.
        relative_path = os.path.join(class_folder, image_file).replace("\\", "/") # Force les slashs pour compatibilité
        
        # --- 2. INFERENCE YOLO ---
        # conf=0.4 : On garde les détections avec >40% de confiance
        # classes=[0] : On ne garde que la classe "personne" (votre main)
        results = model.predict(full_path, save=False, conf=0.4, classes=[0], verbose=False)
        
        for result in results:
            # On vérifie si une boîte a été trouvée
            if len(result.boxes) > 0:
                # On prend la boîte avec le meilleur score (la première)
                box = result.boxes[0]
                
                # Coordonnées YOLO (x_min, y_min, x_max, y_max)
                x_min, y_min, x_max, y_max = box.xyxy[0].tolist()
                
                # Conversion pour Edge Impulse (x, y, largeur, hauteur)
                x = int(x_min)
                y = int(y_min)
                w = int(x_max - x_min)
                h = int(y_max - y_min)
                
                # Ajout au JSON (Clé = chemin relatif)
                if relative_path not in ei_labels["boundingBoxes"]:
                    ei_labels["boundingBoxes"][relative_path] = []
                
                ei_labels["boundingBoxes"][relative_path].append({
                    "label": label,
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h
                })
                
                # Ajout au CSV
                csv_data.append([image_file, label, x, y, w, h, relative_path])

# --- 3. SAUVEGARDE DES FICHIERS ---

# Export JSON (Fichier clé pour l'import auto)
json_path = os.path.join(dataset_dir, "bounding_boxes.labels")
with open(json_path, 'w') as f:
    json.dump(ei_labels, f, indent=4)

# Export CSV
df = pd.DataFrame(csv_data, columns=['filename', 'label', 'x', 'y', 'width', 'height', 'path'])
csv_path = "bounding_boxes.csv"
df.to_csv(csv_path, index=False)

print("\n--- TERMINE ---")
print(f"1. Fichier JSON généré dans : {json_path}")
print(f"   (C'est ce fichier que Edge Impulse utilisera pour placer les boîtes automatiquement)")
print(f"2. Fichier CSV généré : {csv_path} (Pour votre vérification)")

Traitement du dossier : G.class -> Label : G
Traitement du dossier : H.class -> Label : H
Traitement du dossier : I.class -> Label : I
Traitement du dossier : K.class -> Label : K
Traitement du dossier : L.class -> Label : L
Traitement du dossier : M.class -> Label : M
Traitement du dossier : N.class -> Label : N
Traitement du dossier : O.class -> Label : O
Traitement du dossier : P.class -> Label : P
Traitement du dossier : Q.class -> Label : Q
Traitement du dossier : R.class -> Label : R
Traitement du dossier : S.class -> Label : S
Traitement du dossier : T.class -> Label : T
Traitement du dossier : U.class -> Label : U


KeyboardInterrupt: 

# Vérification des Bounding Boxes

Cette section permet de visualiser les boîtes de délimitation générées sur vos images pour valider la qualité de la détection.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import json

# Charger le fichier de bounding boxes
with open('Data/bounding_boxes.labels', 'r') as f:
    bb_data = json.load(f)

print(f"Total d'images avec bounding boxes détectées: {len(bb_data['boundingBoxes'])}")

# Afficher un échantillon d'images avec leurs boîtes
# On affiche 2 images par rangée
images_list = list(bb_data['boundingBoxes'].items())
total_images = min(len(images_list), 10)  # Afficher max 10 images

fig, axes = plt.subplots((total_images + 1) // 2, 2, figsize=(15, 5 * ((total_images + 1) // 2)))
axes = axes.flatten()

for idx, (image_path, boxes) in enumerate(images_list[:total_images]):
    full_image_path = f"Data/{image_path}"
    
    if not os.path.exists(full_image_path):
        print(f"⚠️  Image introuvable: {full_image_path}")
        continue
    
    # Charger et afficher l'image
    img = Image.open(full_image_path)
    axes[idx].imshow(img)
    axes[idx].set_title(f"Label: {boxes[0]['label'] if boxes else 'N/A'}\n{image_path}", fontsize=10)
    
    # Dessiner les bounding boxes
    for box in boxes:
        x = box['x']
        y = box['y']
        w = box['width']
        h = box['height']
        
        # Créer un rectangle
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
        axes[idx].add_patch(rect)
        
        # Ajouter le label
        axes[idx].text(x, y-5, box['label'], color='red', fontsize=9, weight='bold', 
                      bbox=dict(facecolor='yellow', alpha=0.7))
    
    axes[idx].axis('off')

# Masquer les axes inutilisés
for idx in range(total_images, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Visualisation terminée !")

# Vérification détaillée - Naviguer par classe

In [ ]:
# Fonction pour afficher toutes les images d'une classe
def visualize_class(class_letter):
    """Affiche toutes les images avec bounding boxes d'une classe spécifique"""
    
    class_images = [img for img, boxes in bb_data['boundingBoxes'].items() 
                   if boxes and boxes[0]['label'] == class_letter]
    
    if not class_images:
        print(f"❌ Aucune image trouvée pour la classe {class_letter}")
        return
    
    n_images = len(class_images)
    print(f"\n📸 Classe '{class_letter}': {n_images} image(s) détectée(s)")
    
    # Afficher en grille (4 colonnes max)
    cols = min(4, n_images)
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(16, 4 * rows))
    if n_images == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for idx, image_path in enumerate(class_images):
        full_path = f"Data/{image_path}"
        img = Image.open(full_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f"{image_path}", fontsize=9)
        
        # Dessiner les boîtes
        boxes = bb_data['boundingBoxes'][image_path]
        for box in boxes:
            x, y, w, h = box['x'], box['y'], box['width'], box['height']
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='lime', facecolor='none')
            axes[idx].add_patch(rect)
        
        axes[idx].axis('off')
    
    # Masquer les axes inutilisés
    for idx in range(n_images, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()




In [ ]:
visualize_class('B')
visualize_class('C')
visualize_class('D')
visualize_class('E')
visualize_class('F')
visualize_class('G')
visualize_class('H')
visualize_class('I')
visualize_class('K')